### Parameters for YOLOX Training Command

- **`-n <model>`**: Model name, e.g., `yolox-nano`, `yolox-s`, etc. YOLOX supports various models:
  - `yolox-nano`
  - `yolox-tiny`
  - `yolox-s`
  - `yolox-m`
  - `yolox-l`
  - `yolox-x`
- **`-d`**: Number of GPUs to use (e.g., `1` for a single GPU).
- **`-b`**: Batch size. Adjust based on available GPU memory.
- **`--fp16`**: Enables mixed-precision training.
- **`--cache ram`**: Caches the dataset in RAM for faster data loading.
- **`--ckpt`**: Path to a checkpoint file for fine-tuning, e.g., `/workspace/shared/YOLOX_outputs/yolox_pretrained.pth`.
- **`--resume`**: Resume training from a specific checkpoint.
- **`--start_epoch`**: Specify the starting epoch when resuming training.

### Positional Parameters (at the end of the command)
- **`max_epoch`**: Total number of training epochs (e.g., `300`).
- **`print_interval`**: Frequency of logging progress (e.g., every `50` iterations).
- **`eval_interval`**: Frequency of evaluation during training (e.g., every `30` epochs).
- **`output_dir`**: Path to save training results. Recommended path: `/workspace/shared/YOLOX_outputs` for compatibility with Windows (`C:\shared`).
- **`data_dir`**: Path to the dataset directory, e.g., `shared/your-dataset`.
- **`num_classes`**: Number of object classes to detect. For a single class, set this to `1`.


In [ ]:
!python3 -m yolox.tools.train -expn "yolox_nano" -n yolox-nano -d 1 -b 64 --fp16 -o --cache ram max_epoch 300 print_interval 50 eval_interval 30 data_dir "shared/your-dataset" num_classes 1